In [214]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import datetime
# from fake_useragent import UserAgent
import time
import json

from itertools import combinations
from difflib import SequenceMatcher

# from google.cloud import bigquery
# from google.cloud.bigquery import job
from tqdm import tqdm
import pandas as pd
# import google.auth

from openai import OpenAI
import openai

import konlpy
from konlpy.tag import Kkma, Komoran, Okt, Hannanum

# PROJCECT = 'ballosodeuk'
# credentials, project = google.auth.default(
#     scopes=[
#         "https://www.googleapis.com/auth/drive",
#         "https://www.googleapis.com/auth/bigquery",
#     ]
# )
# bq = bigquery.Client(credentials=credentials,project=PROJCECT)

# key = pd.read_csv('./aikey.csv')['key'].values # OpenAI Secret
okt = Okt()


실험 검색어 >> 

In [3]:
search_df = pd.read_csv(r"C:\Users\Owner\Downloads\240902_cp_sampledata.csv",dtype=str)
search_df.head(2)

,sub_id,product_name,ori_url,product_id,item_id,vendoritemid
0,270f7075-5d2f-3aaf-bdc3-5e2e240608e4,대도갤러리 키높이 올루바 긴문형 12자 장롱세트,https://www.coupang.com/vp/products/100064921?...,100064921,305756598,70316548415
1,6213b1e4-21e3-34ec-bbd3-1f009478fc59,1+1 특가이벤트 춘하추동 흑풍 올림찌,https://www.coupang.com/vp/products/100076920?...,100076920,305787463,3754104624


gpt api > 이것도 메소드로 넣을까??


In [193]:
client = OpenAI(api_key=key[1], organization='org-b1AmHOsZydHwAp7obHQA860F')

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


## 작업

### 목표
>상세
- 기획팀에서 정리한 상품 데이터를 기반으로 매칭률 60% 이상을 맞추는 것이 목표 
- 상품명, 아이템ID, 제품ID, 벤더 ID 피처 컬럼이 있음.
- 상품명의 10자를 서치API로 전달하여 10개의 상품리스트를 얻게 됨.
- 전달한 상품명의 피처컬럼이 상품리스트 10개의 피처리스트에 있으면 성공.

>참조
- 벤더 ID가 없는 상품도 있는데, 그럼 나머지 2개가 맞아야 함.
- 10개 이상의 리스트를 받아서 몇위인지 확인하는 등의 시도도 해보셔도 됨.
- 기획에서 얻은 쿠팡 상품 DB가 잘못된 케이스도 좀 있다고 함.. 일단 로직 돌려보시고, 정 이상하면 검증하기로 해요

>메소드 설명
- get_randQuery : 상품명 추출
- get_options : 쿠팡 상품명 구조 상, "," 이후는 옵션임. 옵션과 상품명을 분기하는 기능.
- query_logic_forward : 검색어 생성, 앞 10자
- query_logic_backward : 검색어 생성, 뒤 10자
- get_morps : 검색어 생성 - 유사도, 어절 및 형태소 추출
- similarity : 검색어 생성 - 유사도, 유사도 추출
- find_top_combinations : 검색어 생성, 어절 조합의 유사도 기준 정렬
- get_json_from_url : 서치API 리퀘스트
- get_parse_itemId : 서치 API 검색 결과 정리를 위한 기능
- set_table : 서치 API 검색 결과 테이블

In [365]:
class coupang_sa():
    def __init__(self,dataset):
        self.search_df = dataset

        self.random_queryset = self.get_randQuery()
        self.query_options = self.get_options()
        self.querymorphs = self.get_morps()
        
    ## 모수 추출
    def get_randQuery(self) -> list : #
        rand = random.randint(1,len(search_df))
        rand_query = list(search_df['product_name'])[rand]
        return rand_query

    ## 옵션 추출 basic
    def get_options(self) -> list:
        split_query = re.split(",",self.random_queryset)
        if len(split_query) == 1:
            return split_query
        else:
            del split_query[0]
            return split_query
            
    def query_logic_forward(self):
        seed = self.query_options[0]
        forward = ''
        # 앞 10자
        for i in range(len(seed)):
            forward += seed[i]

            if len("".join(forward.split())) == 11:
                print(forward, len("".join(forward.split())))

                break

        return forward

    def query_logic_backward(self):
        seed = self.query_options[0]
        backward = ''
        # 뒤 10자
        for i in range(len(seed) -1,-1,-1):
            backward += seed[i]

            if len("".join(backward.split())) == 11:
                backward = backward[::-1]
                print(backward, len("".join(backward.split())))

                break

        return backward

    def get_morps(self) -> list :
        phrases = okt.phrases(self.random_queryset)
        return phrases

    ## 키워드 추출 intermediate
    def similarity(self, a, b) -> float:
        return SequenceMatcher(None, a, b).ratio()
    
    def find_top_combinations(self, max_len=11, top_n=5) -> list[tuple[str,float]]:
        combinations_list = []
        # print(self.querymorphs)
        # print(self.random_queryset)
        for r in range(1, len(self.querymorphs) + 1):
            for combo in combinations(self.querymorphs, r):
                combined_phrase = " ".join(combo).replace(" ", "")
                if len(combined_phrase) <= max_len:
                    sim = self.similarity(self.random_queryset, combined_phrase)
                    combinations_list.append((" ".join(combo), sim))

        # print(111222,self.querymorphs, self.random_queryset)
        combinations_list.sort(key=lambda x: x[1], reverse=True)
        return combinations_list[:top_n]
    
    # 서치api 작동
    def get_json_from_url(self, query) -> json : 
        url = f'https://api.adjoin.co.kr/cou/api.php?code=balsosa1&q={query}&c=10'
        try:
            response = requests.get(url)
            response.raise_for_status()  
            data = response.json() 
            return data
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}") 
        except Exception as err:
            print(f"An error occurred: {err}")
        return None
    
    def get_parse_itemId(self, items,order) -> list :
        if order == 'itemid':
            parsed = re.search(r'itemId%3D(\d+)', items).group(1)
        elif order == 'verdorItemId':
            parsed = re.search(r'vendorItemId%3D(\d+)', items).group(1)
        
        return parsed

    def set_table(self, json_data) -> pd.DataFrame :
        product_name = [str(item['productName']) for item in json_data['data']['productData']]
        product_id = [str(item['productId']) for item in json_data['data']['productData']]
        item_id = [self.get_parse_itemId(item['productUrl'],'itemid') for item in json_data['data']['productData']]
        vendor_id = [self.get_parse_itemId(item['productUrl'],'verdorItemId') for item in json_data['data']['productData']]

        api_df = pd.DataFrame({
            'product_name' : product_name,
            'product_id' : product_id,
            'item_id' : item_id,
            'vendor_id' : vendor_id})
        
        return api_df
    


In [366]:
dataset = pd.read_csv(r"C:\Users\Owner\Downloads\240902_cp_sampledata.csv",dtype=str)

cp = coupang_sa(dataset)

cp.query_logic()


크리스찬 [Christ 11
T217 네이비 체크 셔츠 11


['크리스찬 [Christ', 'T217 네이비 체크 셔츠']

In [233]:
type(cp.similarity('감사합니다','감사다'))

float